In [1]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model

np.random.seed(13)

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [2]:
C = 5 # number of campaigns
U = 100 # number of customers.
H = 2 # number of channels.
D = 7 # number of planning days.
I = 3 # number of quota categories.
P = 10 # number of priority categories.

In [3]:
#C = 10 # number of campaigns
#U = 10000 # number of customers.
#H = 3 # number of channels.
#D = 7 # number of planning days.
#I = 3 # number of quota categories.
#P = 10 # number of priority categories.

#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
e_cu = np.random.choice(2,(C, U))
#e_cu = np.ones((C, U), dtype='int8')
e_cu

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        1, 0, 0, 0, 0,

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [5]:
q_ic = np.random.choice(2, (I,C))
#q_ic = np.zeros((I,C), dtype='int8')
q_ic

array([[1, 1, 1, 0, 0],
       [1, 1, 0, 1, 0],
       [0, 1, 1, 1, 1]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [6]:
r_p = np.random.choice(100, P)
#r_p = np.ones(P, dtype='int8')
rp_c = [r_p[r] for r in np.random.choice(P, C)]
rp_c

[41, 37, 45, 47, 52]

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [7]:
b = 7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [8]:
k = 2

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [9]:
l_c = np.random.choice([2,3,4],C)
l_c

array([2, 4, 4, 3, 3])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [10]:
m_i = np.random.choice([10,15,18],I)
n_i = np.random.choice([2,3,4],I)
#m_i = np.ones((I), dtype='int8')*10
#n_i = np.ones((I), dtype='int8')*10
(m_i, n_i)

(array([15, 15, 10]), array([3, 3, 3]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [11]:
t_hd = np.random.choice([70], (H, D))

### Model

In [12]:
mdl = Model(name='Campaign Optimization')

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [13]:
X_cuhd = {(c,u,h,d): mdl.binary_var(f"X_c:{c}_u:{u}_h:{h}_d:{d}")
            for c in range(0,C)
            for u in range(0,U) 
            for h in range(0,H)
            for d in range(0,D)}
"Done"

'Done'

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

In [14]:
mdl.maximize(mdl.sum([X_cuhd[(c,u,h,d)] * rp_c[c]
                  for p in range(0,P)
                  for c in range(0,C)
                  for u in range(0,U) 
                  for h in range(0,H) 
                  for d in range(0,D)]))

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall c \in C,\forall u \in U,\forall h \in H,\forall d \in D
$$

In [15]:
mdl.add_constraints(
    (X_cuhd[(c,u,h,d)] <= e_cu[c,u]
    for c in range(0,C)
    for u in range(0,U) 
    for h in range(0,H) 
    for d in range(0,D))
)
"Done"

'Done'

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [16]:
mdl.add_constraints(
    ((mdl.sum(X_cuhd[(c,u,h,d)] for h in range(0,H)) <= 1)
    for c in range(0,C)
    for u in range(0,U) 
    for d in range(0,D))
)
"Done"

'Done'

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [17]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)] 
               for d in range(0,D) 
               for c in range(0,C) 
               for h in range(0,H)) <= b)
        for u in range(0,U)))
"Done"

'Done'

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [18]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for c in range(0,C) 
               for h in range(0,H)) <= k)
        for d in range(0,D)
        for u in range(0,U)))
"Done"

'Done'

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [19]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for h in range(0,H) 
               for d in range(0,D)) <= l_c[c] )
        for c in range(0,C)
        for u in range(0,U)))
"Done"

'Done'

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [20]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*q_ic[i,c]
               for c in range(0,C)
               for h in range(0,H) 
               for d in range(0,D)) <= m_i[i])
        for u in range(0,U)
        for i in range(0,I)))
"Done"

'Done'

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [21]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*q_ic[i,c]
               for c in range(0,C) 
               for h in range(0,H)) <= n_i[i])
        for u in range(0,U)
        for d in range(0,D)
        for i in range(0,I)))
"Done"

'Done'

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [22]:
mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]
               for u in range(0,U) 
               for c in range(0,C)) <= t_hd[h,d])
        for h in range(0,H)
        for d in range(0,D)))
"Done"

'Done'

### Solution

In [23]:
solution = mdl.solve(log_output=True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Found incumbent of value 0.000000 after 0.00 sec. (0.26 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 11842 rows and 3612 columns.
Reduced MIP has 2372 rows, 3388 columns, and 15596 nonzeros.
Reduced MIP has 3388 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (17.39 ticks)
Probing time = 0.04 sec. (0.50 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 2372 rows, 3388 columns, and 15596 nonzeros.
Reduced MIP has 3388 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (16.23 ticks)
Probing time = 0.02 sec. (0.62 ticks)
Clique table members: 1694.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.03 sec. (10.16 ticks)

       

In [24]:
solution

docplex.mp.solution.SolveSolution(obj=268420,values={X_c:0_u:7_h:0_d:3:1..

In [25]:
print(solution)

solution for: Campaign Optimization
objective: 268420
X_c:0_u:7_h:0_d:3=1
X_c:0_u:10_h:0_d:0=1
X_c:0_u:10_h:0_d:1=1
X_c:0_u:17_h:0_d:0=1
X_c:0_u:17_h:0_d:1=1
X_c:0_u:18_h:0_d:0=1
X_c:0_u:18_h:0_d:1=1
X_c:0_u:20_h:0_d:0=1
X_c:0_u:20_h:0_d:1=1
X_c:0_u:21_h:0_d:0=1
X_c:0_u:21_h:0_d:1=1
X_c:0_u:27_h:0_d:0=1
X_c:0_u:27_h:0_d:1=1
X_c:0_u:28_h:0_d:0=1
X_c:0_u:28_h:0_d:1=1
X_c:0_u:31_h:0_d:0=1
X_c:0_u:31_h:0_d:1=1
X_c:0_u:36_h:0_d:2=1
X_c:0_u:36_h:0_d:4=1
X_c:0_u:43_h:0_d:2=1
X_c:0_u:43_h:0_d:3=1
X_c:0_u:45_h:0_d:0=1
X_c:0_u:45_h:0_d:1=1
X_c:0_u:46_h:0_d:0=1
X_c:0_u:46_h:0_d:1=1
X_c:0_u:51_h:0_d:0=1
X_c:0_u:51_h:0_d:1=1
X_c:0_u:52_h:0_d:1=1
X_c:0_u:54_h:0_d:0=1
X_c:0_u:54_h:0_d:1=1
X_c:0_u:56_h:0_d:5=1
X_c:0_u:61_h:0_d:3=1
X_c:0_u:61_h:0_d:4=1
X_c:0_u:64_h:0_d:5=1
X_c:0_u:65_h:0_d:3=1
X_c:0_u:65_h:0_d:4=1
X_c:0_u:67_h:0_d:0=1
X_c:0_u:67_h:0_d:1=1
X_c:0_u:72_h:0_d:0=1
X_c:0_u:72_h:0_d:3=1
X_c:0_u:76_h:0_d:2=1
X_c:0_u:76_h:0_d:3=1
X_c:0_u:82_h:0_d:0=1
X_c:0_u:82_h:0_d:1=1
X_c:0_u:83_h:0_d:0=1
X_